In [2]:
import numpy as np 
import pandas as pd
import os
from typing import Optional, Tuple

import torch
from torch import nn

Transformation of Coordinates UTM

In [ ]:
from pyproj import Proj, transform

# UTM Zone for southern Quebec (Zone 18T)
utm_proj = Proj(proj='utm', zone=18, ellps='WGS84')
latlon_proj = Proj(proj='latlong', datum='WGS84')

# Example of conversion
# lat, lon = transform(utm_proj, latlon_proj, x_utm, y_utm)

lat, lon = 

Aggregate Rainfall Data (Hourly to Daily)

In [5]:
# # Specify the folder containing the CSV files
# folder_path = '/Users/danilgarmaev/Documents/Fall 2024 M.Sc. AI/Ericsson/data/Hackathon_data/Hackathon_rainfall_train_set'

# # Dictionary to store dataframes with coordinates as keys
# data_frames_dict = {}

# # Loop through all files in the folder
# for filename in os.listdir(folder_path):
#     # Check if the file ends with '.csv' and follows the format 'Txxxx.csv'
#     if filename.startswith('T'):
#         # Extract the x and y coordinates from the filename
#         x = int(filename[1:3])  # First 2 digits after 'T'
#         y = int(filename[3:5])  # Next 2 digits after 'T'

#         # Construct the full file path
#         file_path = os.path.join(folder_path, filename)

#         # Read the CSV file into a dataframe
#         df = pd.read_csv(file_path)

#         # Store the dataframe in the dictionary with (x, y) as the key
#         data_frames_dict[(x, y)] = df

#         print(f'Added dataframe for key ({x}, {y})')

# # Now `data_frames_dict` contains the dataframes with their (x, y) coordinates as keys
# print("All CSV files have been processed.")

Added dataframe for key (2, 8)
Added dataframe for key (6, 9)
Added dataframe for key (7, 2)
Added dataframe for key (3, 3)
Added dataframe for key (6, 1)
Added dataframe for key (9, 7)
Added dataframe for key (9, 12)
Added dataframe for key (8, 11)
Added dataframe for key (8, 4)
Added dataframe for key (9, 8)
Added dataframe for key (2, 12)
Added dataframe for key (2, 7)
Added dataframe for key (6, 6)
Added dataframe for key (3, 4)
Added dataframe for key (3, 11)
Added dataframe for key (7, 10)
Added dataframe for key (7, 5)
Added dataframe for key (8, 3)
Added dataframe for key (7, 4)
Added dataframe for key (7, 11)
Added dataframe for key (3, 10)
Added dataframe for key (3, 5)
Added dataframe for key (6, 12)
Added dataframe for key (6, 7)
Added dataframe for key (2, 6)
Added dataframe for key (9, 1)
Added dataframe for key (8, 2)
Added dataframe for key (9, 9)
Added dataframe for key (2, 1)
Added dataframe for key (3, 2)
Added dataframe for key (7, 3)
Added dataframe for key (8, 5)


In [ ]:
# Specify the folder containing the CSV files
rainfall_files = '/Users/danilgarmaev/Documents/Fall 2024 M.Sc. AI/Ericsson/data/Hackathon_data/Hackathon_rainfall_train_set'

# Dictionary to store dataframes with coordinates as keys
rainfall_data = {}
for filename in os.listdir(rainfall_files):
    # Extract the point_id from the filename (e.g., T0101R)
    # point_id = file.split('\\')[-1].split('R')[0] + 'R'  # Extracting T0101R from T0101R.csv_train.csv

    if filename.startswith('T'):
        # Extract the x and y coordinates from the filename
        name  = filename[:5]
        file_path = os.path.join(rainfall_files, filename)        
        # Store the dataframe in the dictionary with (x, y) as the key
        rainfall_data[name] = pd.read_csv(file_path)

    rainfall_data[name] = pd.read_csv(file_path)
    if 'time' in rainfall_data[name].columns:
            rainfall_data[name]['time'] = pd.to_datetime(rainfall_data[name]['time'])

    if 'rate' in rainfall_data[name].columns:
        # Group by date and fill null values
        rainfall_data[name]['rate'] = rainfall_data[name].groupby(rainfall_data[name]['time'].dt.date)['rate'].transform(lambda x: x.fillna(x.mean()))


In [11]:
import tensorflow

2024-10-05 16:50:46.928209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Charger les données
df = pd.read_csv('/Users/danilgarmaev/Documents/Fall 2024 M.Sc. AI/Ericsson/data/Hackathon_data/Hackathon_water_level_train_set/water_level_train.csv', parse_dates=['time'])
df.set_index('time', inplace=True)

# Trier l'index et rééchantillonner les données sur une base journalière
df = df.sort_index().resample('D').mean()

# Identifier les dates manquantes
missing_dates = pd.date_range(start=df.index.min(), end=df.index.max()).difference(df.index)

# Normaliser les données
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Préparer les données pour le LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 7
X, y = create_sequences(scaled_data, seq_length)

# Créer et entraîner le modèle LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X, y, epochs=100, batch_size=32, verbose=0)

# Prédire les valeurs manquantes
predictions = []
for date in missing_dates:
    input_data = df.loc[:date - pd.Timedelta(days=1)].tail(seq_length)
    if len(input_data) < seq_length:
        continue
    scaled_input = scaler.transform(input_data)
    X_pred = scaled_input.reshape((1, seq_length, 1))
    scaled_pred = model.predict(X_pred)
    pred = scaler.inverse_transform(scaled_pred)
    predictions.append((date, pred[0][0]))

# Créer un DataFrame avec les prédictions
predictions_df = pd.DataFrame(predictions, columns=['time', 'Water level(m)'])
predictions_df.set_index('time', inplace=True)

# Combiner les données originales et les prédictions
result_df = pd.concat([df, predictions_df])
result_df = result_df.sort_index()

print(result_df)